# Implement and test prediciton model

## Prerequisites

In [10]:
# Prerequisites
from tabulate import tabulate
from transformers import pipeline
import json
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
import time
import math

# get candidate labels
with open("packing_label_structure.json", "r") as file:
    candidate_labels = json.load(file)
keys_list = list(candidate_labels.keys())

# Load test data (in list of dictionaries)
with open("test_data.json", "r") as file:
    packing_data = json.load(file)
# Extract all trip descriptions and trip_types
trip_descriptions = [trip['description'] for trip in packing_data]
trip_types = [trip['trip_types'] for trip in packing_data]

# Access the first trip description
first_trip = trip_descriptions[0]
# Get the packing list for the secondfirst trip
first_trip_type = trip_types[0]

# print(f"First trip: {first_trip} \n")
# print(f"Trip type: {first_trip_type}")

**All trip descriptions**

In [11]:
for i, item in enumerate(trip_descriptions):
    print(i, ".", item, "\n")
    for elem in trip_types[i]:
        print(elem)
    print("\n")

0 . I am planning a trip to Greece with my boyfriend, where we will visit two islands. We have booked an apartment on each island for a few days and plan to spend most of our time relaxing. Our main goals are to enjoy the beach, try delicious local food, and possibly go on a hike—if it’s not too hot. We will be relying solely on public transport. We’re in our late 20s and traveling from the Netherlands. 

beach vacation
['swimming', 'going to the beach', 'relaxing', 'hiking']
warm destination / summer
lightweight (but comfortable)
casual
indoor
no own vehicle
no special conditions to consider
7+ days


1 . We are a couple in our thirties traveling to Vienna for a three-day city trip. We’ll be staying at a friend’s house and plan to explore the city by sightseeing, strolling through the streets, visiting markets, and trying out great restaurants and cafés. We also hope to attend a classical music concert. Our journey to Vienna will be by train. 

city trip
['sightseeing']
variable weath

**Functions**

In [12]:
# function that returns pandas data frame with predictions

cut_off = 0.5  # used to choose which activities are relevant

def pred_trip(model_name, trip_descr, trip_type, cut_off):
    classifier = pipeline("zero-shot-classification", model=model_name)
    # Create an empty DataFrame with specified columns
    df = pd.DataFrame(columns=['superclass', 'pred_class'])
    for i, key in enumerate(keys_list):
        if key == 'activities':
            result = classifier(trip_descr, candidate_labels[key], multi_label=True)
            indices = [i for i, score in enumerate(result['scores']) if score > cut_off]
            classes = [result['labels'][i] for i in indices]
        else:
            result = classifier(trip_descr, candidate_labels[key])
            classes = result["labels"][0]
        print(result)
        print(classes)
        print(i)
        df.loc[i] = [key, classes]
    df['true_class'] = trip_type
    return df

In [13]:
# function for accuracy, perc true classes identified and perc wrong pred classes

def perf_measure(df):
    df['same_value'] = df['pred_class'] == df['true_class']
    correct = sum(df.loc[df.index != 1, 'same_value'])
    total = len(df['same_value'])
    accuracy = correct/total
    pred_class = df.loc[df.index == 1, 'pred_class'].iloc[0]
    true_class = df.loc[df.index == 1, 'true_class'].iloc[0]
    correct = [label for label in pred_class if label in true_class]
    num_correct = len(correct)
    correct_perc = num_correct/len(true_class)
    num_pred = len(pred_class)
    if num_pred == 0:
        wrong_perc = math.nan
    else:
        wrong_perc = (num_pred - num_correct)/num_pred
    df_perf = pd.DataFrame({
    'accuracy': [accuracy],
    'true_ident': [correct_perc],
    'false_pred': [wrong_perc]
    })
    return(df_perf)

## Make predictions for many models and trip descriptions

Provide a list of candidate models and apply them to the test data.

In [5]:
# List of Hugging Face model names
model_names = [
    "facebook/bart-large-mnli",
    "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli",
    ##"cross-encoder/nli-deberta-v3-base",
    "cross-encoder/nli-deberta-v3-large",
    "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli",
    ##"joeddav/bart-large-mnli-yahoo-answers",
    "MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli",
    "MoritzLaurer/deberta-v3-large-zeroshot-v2.0",
    "valhalla/distilbart-mnli-12-1",
    #"joeddav/xlm-roberta-large-xnli" # keeps giving errors
]

# Apply each model to the test data
for model_name in model_names:
    print(f"\nUsing model: {model_name}")
    result_list = []
    performance = pd.DataFrame(columns=['accuracy', 'true_ident', 'false_pred'])
    
    start_time = time.time()
    for i in range(len(trip_descriptions)):
        current_trip = trip_descriptions[i]
        current_type = trip_types[i]
        df = pred_trip(model_name, current_trip, current_type, cut_off = 0.5)
        print(df)
        # accuracy, perc true classes identified and perc wrong pred classes
        performance = pd.concat([performance, perf_measure(df)])
        print(performance)
        
        result_list.append(df)
    end_time = time.time()
    elapsed_time = end_time - start_time
    # Extract "same_value" column from each DataFrame
    sv_columns = [df['same_value'] for df in result_list]  # 'same' needs to be changed
    sv_columns.insert(0, result_list[0]['superclass'])
    # Combine into a new DataFrame (columns side-by-side)
    sv_df = pd.concat(sv_columns, axis=1)
    print(sv_df)
    # Compute accuracy per superclass (row means of same_value matrix excluding the first column)
    row_means = sv_df.iloc[:, 1:].mean(axis=1)
    df_row_means = pd.DataFrame({
        'superclass': sv_df['superclass'],
        'accuracy': row_means
    })
    print(df_row_means)
    # Compute performance measures per trip (mean for each column of performance table)
    column_means = performance.mean()
    print(column_means)
    # save results
    model = model_name.replace("/", "-")
    model_result = {
        'model': model,
        'predictions': result_list,
        'performance': performance,
        'perf_summary': column_means,
        'perf_superclass': df_row_means,
        'elapsed_time': elapsed_time
    }
    # File path with folder
    filename = os.path.join('results', f'{model}_results.pkl')
    # Save the object
    with open(filename, 'wb') as f:
        pickle.dump(model_result, f)






Using model: cross-encoder/nli-deberta-v3-base


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'sequence': 'I am planning a trip to Greece with my boyfriend, where we will visit two islands. We have booked an apartment on each island for a few days and plan to spend most of our time relaxing. Our main goals are to enjoy the beach, try delicious local food, and possibly go on a hike—if it’s not too hot. We will be relying solely on public transport. We’re in our late 20s and traveling from the Netherlands.', 'labels': ['micro-adventure / weekend trip', 'digital nomad trip', 'beach vacation', 'festival trip', 'city trip', 'cultural exploration', 'road trip (car/camper)', 'camping trip (wild camping)', 'long-distance hike / thru-hike', 'hut trek (winter)', 'ski tour / skitour', 'snowboard / splitboard trip', 'nature escape', 'yoga / wellness retreat', 'hut trek (summer)', 'camping trip (campground)'], 'scores': [0.9722680449485779, 0.007802918087691069, 0.0075571718625724316, 0.0022959215566515923, 0.0021305829286575317, 0.001222927705384791, 0.0009879637509584427, 0.0008052966441

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'sequence': 'We are a couple in our thirties traveling to Vienna for a three-day city trip. We’ll be staying at a friend’s house and plan to explore the city by sightseeing, strolling through the streets, visiting markets, and trying out great restaurants and cafés. We also hope to attend a classical music concert. Our journey to Vienna will be by train.', 'labels': ['city trip', 'micro-adventure / weekend trip', 'festival trip', 'digital nomad trip', 'cultural exploration', 'ski tour / skitour', 'hut trek (summer)', 'hut trek (winter)', 'camping trip (campground)', 'long-distance hike / thru-hike', 'beach vacation', 'snowboard / splitboard trip', 'camping trip (wild camping)', 'road trip (car/camper)', 'nature escape', 'yoga / wellness retreat'], 'scores': [0.7273069024085999, 0.16893576085567474, 0.0374605655670166, 0.013699190691113472, 0.010825436562299728, 0.009758710861206055, 0.007331428118050098, 0.004068635869771242, 0.003749603172764182, 0.003352535655722022, 0.0031235795468

...

{'sequence': 'We will go to Sweden in the winter, to go for a yoga and sauna/wellness retreat. I prefer lightweight packing and also want clothes to go for fancy dinners and maybe on a winter hike. We stay in hotels.', 'labels': ['yoga / wellness retreat', 'micro-adventure / weekend trip', 'nature escape', 'long-distance hike / thru-hike', 'cultural exploration', 'ski tour / skitour', 'hut trek (winter)', 'city trip', 'beach vacation', 'road trip (car/camper)', 'hut trek (summer)', 'festival trip', 'camping trip (wild camping)', 'snowboard / splitboard trip', 'camping trip (campground)', 'digital nomad trip'], 'scores': [0.2606178820133209, 0.08046605437994003, 0.06843063235282898, 0.059995006769895554, 0.059469204396009445, 0.05855844169855118, 0.055849116295576096, 0.047889091074466705, 0.045752931386232376, 0.04353173449635506, 0.04008011892437935, 0.038273148238658905, 0.03779057413339615, 0.03708784282207489, 0.03450765833258629, 0.03170054778456688]}
yoga / wellness retreat
0
{'s

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'sequence': 'I go on a skitouring trip where we also make videos/photos and the destination is Japan. Mainly sports clothes and isolation are needed (it is winter). We stay in a guesthouse. It will be 10 days.', 'labels': ['ski tour / skitour', 'micro-adventure / weekend trip', 'cultural exploration', 'road trip (car/camper)', 'city trip', 'snowboard / splitboard trip', 'long-distance hike / thru-hike', 'digital nomad trip', 'hut trek (winter)', 'festival trip', 'hut trek (summer)', 'camping trip (wild camping)', 'nature escape', 'camping trip (campground)', 'beach vacation', 'yoga / wellness retreat'], 'scores': [0.1734267771244049, 0.09546443074941635, 0.08052344620227814, 0.07985977828502655, 0.06759097427129745, 0.06277172267436981, 0.054686594754457474, 0.05254476144909859, 0.0503639318048954, 0.044427838176488876, 0.043578390032052994, 0.043481748551130295, 0.04278255254030228, 0.04047577455639839, 0.038092684000730515, 0.029928630217909813]}
ski tour / skitour
0
{'sequence': 'I

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'sequence': 'We plan a wild camping trip with activities such as snorkeling, kayaking and canoeing. It is a warm place and we want to bring little stuff. We stay in tents and hammocks and travel with a car, it will be 3 days.', 'labels': ['camping trip (wild camping)', 'camping trip (campground)', 'micro-adventure / weekend trip', 'road trip (car/camper)', 'nature escape', 'beach vacation', 'long-distance hike / thru-hike', 'cultural exploration', 'festival trip', 'ski tour / skitour', 'hut trek (summer)', 'city trip', 'hut trek (winter)', 'snowboard / splitboard trip', 'digital nomad trip', 'yoga / wellness retreat'], 'scores': [0.24850936233997345, 0.14938758313655853, 0.09667350351810455, 0.08339172601699829, 0.05508505925536156, 0.051402922719717026, 0.044685814529657364, 0.03972415253520012, 0.03578963875770569, 0.03319094330072403, 0.03277123346924782, 0.029747819527983665, 0.02797492779791355, 0.027441974729299545, 0.023192668333649635, 0.021030662581324577]}
camping trip (wild

## Load and compare results

In [14]:
# Folder where .pkl files are saved
results_dir = 'results/before'

# Dictionary to store all loaded results
all_results = {}

# Loop through all .pkl files in the folder
for filename in os.listdir(results_dir):
    if filename.endswith('.pkl'):
        model_name = filename.replace('_results.pkl', '')  # Extract model name
        file_path = os.path.join(results_dir, filename)
        
        # Load the result
        with open(file_path, 'rb') as f:
            result = pickle.load(f)
            all_results[model_name] = result

# Compare performance across models
for model, data in all_results.items():
    print(f"Model: {model}")
    print(f"Performance Summary:\n{data['perf_summary']}")
    print("-" * 40)

# Compare performance across models
for model, data in all_results.items():
    print(f"Model: {model}")
    print(f"Performance Summary:\n{data['perf_superclass']}")
    print("-" * 40)

Model: cross-encoder-nli-deberta-v3-base
Performance Summary:
accuracy      0.444444
true_ident    0.533333
false_pred    0.712500
dtype: float64
----------------------------------------
Model: joeddav-bart-large-mnli-yahoo-answers
Performance Summary:
accuracy      0.344444
true_ident    0.650000
false_pred    0.553792
dtype: float64
----------------------------------------
Model: cross-encoder-nli-deberta-v3-large
Performance Summary:
accuracy      0.466667
true_ident    0.566667
false_pred    0.541667
dtype: float64
----------------------------------------
Model: MoritzLaurer-DeBERTa-v3-large-mnli-fever-anli-ling-wanli
Performance Summary:
accuracy      0.566667
true_ident    0.841667
false_pred    0.546667
dtype: float64
----------------------------------------
Model: MoritzLaurer-mDeBERTa-v3-base-mnli-xnli
Performance Summary:
accuracy      0.466667
true_ident    0.408333
false_pred    0.481250
dtype: float64
----------------------------------------
Model: MoritzLaurer-deberta-v3-

**Identify trips that are difficult to predict**

Per model

In [15]:
def get_difficult_trips(model_result, cut_off = 0.6):
    # model_result is a dict with dict_keys(['model', 'predictions', 
    # 'performance', 'perf_summary', 'perf_superclass', 'elapsed_time'])
    # get performance dataframe and repair index
    df = model_result['performance'].reset_index(drop=True)
    # find index of trips whose accuracy is below cut_off
    index_result = df[df['accuracy'] < cut_off].index
    return(index_result)

# dictionary of trips that have accuracy below cut_off default
difficult_trips_dict = {}
for model, data in all_results.items():
    difficult_trips_dict[data["model"]] = get_difficult_trips(data)

for key, value in difficult_trips_dict.items():
    print(f"{key}: {value}\n")

cross-encoder-nli-deberta-v3-base: Index([0, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')

joeddav-bart-large-mnli-yahoo-answers: RangeIndex(start=0, stop=10, step=1)

cross-encoder-nli-deberta-v3-large: Index([0, 1, 2, 3, 4, 6, 7, 8, 9], dtype='int64')

MoritzLaurer-DeBERTa-v3-large-mnli-fever-anli-ling-wanli: Index([2, 5, 6, 7, 8, 9], dtype='int64')

MoritzLaurer-mDeBERTa-v3-base-mnli-xnli: RangeIndex(start=0, stop=10, step=1)

MoritzLaurer-deberta-v3-large-zeroshot-v2.0: Index([0, 1, 2, 3, 4, 5, 6, 7, 9], dtype='int64')

facebook-bart-large-mnli: RangeIndex(start=0, stop=10, step=1)

valhalla-distilbart-mnli-12-1: Index([0, 1, 2, 3, 4, 7, 9], dtype='int64')

MoritzLaurer-DeBERTa-v3-base-mnli-fever-anli: Index([0, 2, 3, 4, 6, 7], dtype='int64')



For all models

In [16]:
# Which trips are difficult for all models
common = set.intersection(*(set(v) for v in difficult_trips_dict.values()))
for index in common:
    print(index, ".", trip_descriptions[index], "\n")
    for item in trip_types[index]:
        print(item)
    print("\n")

2 . My partner and I are traveling to the Netherlands and Germany to spend Christmas with our family. We are in our late twenties and will start our journey with a two-hour flight to the Netherlands. From there, we will take a 5.5-hour train ride to northern Germany. 

city trip
['relaxing']
cold destination / winter
lightweight (but comfortable)
casual
indoor
no own vehicle
no special conditions to consider
7+ days


7 . We will go to Sweden in the winter, to go for a yoga and sauna/wellness retreat. I prefer lightweight packing and also want clothes to go for fancy dinners and maybe on a winter hike. We stay in hotels. 

yoga / wellness retreat
['hut-to-hut hiking', 'yoga']
cold destination / winter
lightweight (but comfortable)
formal (business trip)
sleeping in a tent
no own vehicle
avalanche-prone terrain
7 days




**Identify superclasses that are difficult to predict**

Per model

In [17]:
def get_difficult_superclasses(model_result, cut_off = 0.6):
    # model_result is a dict with dict_keys(['model', 'predictions', 
    # 'performance', 'perf_summary', 'perf_superclass', 'elapsed_time'])
    df = model_result["perf_superclass"]
    # find superclass whose accuracy is below cut_off
    diff_spc = list(df[df['accuracy'] < cut_off]["superclass"])
    return(diff_spc)

# make dictionary of superclasses that have accuracy below cut_off default
difficult_superclass_dict = {}
for model, data in all_results.items():
    difficult_superclass_dict[data["model"]] = get_difficult_superclasses(data)

for key, value in difficult_superclass_dict.items():
    print(f"{key}: {value}\n")

cross-encoder-nli-deberta-v3-base: ['activities', 'climate_or_season', 'style_or_comfort', 'special_conditions']

joeddav-bart-large-mnli-yahoo-answers: ['activities', 'climate_or_season', 'style_or_comfort', 'dress_code', 'accommodation', 'transportation', 'special_conditions']

cross-encoder-nli-deberta-v3-large: ['activities', 'climate_or_season', 'style_or_comfort', 'transportation', 'special_conditions']

MoritzLaurer-DeBERTa-v3-large-mnli-fever-anli-ling-wanli: ['activities', 'style_or_comfort', 'special_conditions']

MoritzLaurer-mDeBERTa-v3-base-mnli-xnli: ['activities', 'style_or_comfort', 'accommodation', 'special_conditions', 'trip_length_days']

MoritzLaurer-deberta-v3-large-zeroshot-v2.0: ['activities', 'climate_or_season', 'style_or_comfort', 'accommodation', 'special_conditions']

facebook-bart-large-mnli: ['activities', 'style_or_comfort', 'accommodation', 'special_conditions']

valhalla-distilbart-mnli-12-1: ['activities', 'style_or_comfort', 'accommodation', 'special_

For all models

In [18]:
# Which trips are difficult for all models
common = set.intersection(*(set(v) for v in difficult_superclass_dict.values()))
print(common)

{'style_or_comfort', 'activities', 'special_conditions'}


In [19]:
# Look at particular predicitons in detail
# print(all_results["joeddav-bart-large-mnli-yahoo-answers"])

**Comparing models**

In [24]:
# Make table of 'perf_summary' for all models inlcude time elapsed
print(type(all_results))
print(type(all_results["MoritzLaurer-DeBERTa-v3-base-mnli-fever-anli"]))
print(all_results["MoritzLaurer-DeBERTa-v3-base-mnli-fever-anli"].keys())
print(type(all_results["MoritzLaurer-DeBERTa-v3-base-mnli-fever-anli"]["perf_summary"]))
print(all_results["MoritzLaurer-DeBERTa-v3-base-mnli-fever-anli"]["perf_summary"])
print(all_results["MoritzLaurer-DeBERTa-v3-base-mnli-fever-anli"]["perf_summary"]["accuracy"])
# make empty data frame
# fill in for loop with perf_summary per model


# Make ranking from that table for each category


<class 'dict'>
<class 'dict'>
dict_keys(['model', 'predictions', 'performance', 'perf_summary', 'perf_superclass', 'elapsed_time'])
<class 'pandas.core.series.Series'>
accuracy      0.522222
true_ident    0.841667
false_pred    0.572381
dtype: float64
0.5222222222222223


# Use gradio for user input

In [66]:
# use model with gradio
from transformers import pipeline
import gradio as gr

# make a function for what I am doing
def classify(text):
    df = pd.DataFrame(columns=['Superclass', 'class'])
    for i, key in enumerate(keys_list):
        # Run the classification (ca 30 seconds classifying)
        if key == 'activities':
            result = classifier(text, candidate_labels[key], multi_label=True)
            classes = [result['labels'][i] for i in indices]
        else:
            result = classifier(text, candidate_labels[key])
            classes = result["labels"][0]
        print(i)
        df.loc[i] = [key, classes]

    return df

demo = gr.Interface(
    fn=classify,
    inputs="text",
    outputs="dataframe",
    title="Zero-Shot Classification",
    description="Enter a text describing your trip",
)

# Launch the Gradio app
if __name__ == "__main__":
    demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://aa06d5d85ffadaa92b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8


Use model with gradio

In [4]:
# Define the Gradio interface
def classify(text):
    return classifier(text, class_labels)

demo = gr.Interface(
    fn=classify,
    inputs="text",
    outputs="json",
    title="Zero-Shot Classification",
    description="Enter a text describing your trip",
)

# Launch the Gradio app
if __name__ == "__main__":
    demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://0f70ba5369d721cf8f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
